## Deckblatt-Generator

In [5]:
import os
import random
from PIL import Image

def generate_collage(root_dir, output_dir, output_size=(3840, 2160), min_image_size=(50, 50), max_images=200):
    collage = Image.new('RGB', output_size)

    # Get all jpg file paths
    image_paths = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith('.jpg'):
                image_paths.append(os.path.join(dirpath, filename))

    if not image_paths:
        print("Keine Bilder gefunden!")
        return

    # Limit the number of images based on max_images
    if len(image_paths) > max_images:
        image_paths = random.sample(image_paths, max_images)

    for image_path in image_paths:
        with Image.open(image_path) as img:
            # Resize the image to fit completely into the collage
            scale_factor = random.uniform(0.5, 1.0)  # Resize factor to ensure overlap and coverage
            new_size = (max(min_image_size[0], int(img.size[0] * scale_factor)),
                        max(min_image_size[1], int(img.size[1] * scale_factor)))
            
            img = img.resize(new_size, Image.LANCZOS)

            # Random position
            max_x = max(0, output_size[0] - new_size[0])
            max_y = max(0, output_size[1] - new_size[1])
            pos_x = random.randint(0, max_x)
            pos_y = random.randint(0, max_y)

            # Paste the image
            collage.paste(img, (pos_x, pos_y), img if img.mode == 'RGBA' else None)

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Save the collage
    output_path = os.path.join(output_dir, f"collage_{random.randint(1000, 9999)}.jpg")
    collage.save(output_path)
    print(f"Collage saved as {output_path}")

if __name__ == "__main__":
    # Root directory for images, output directory, and max images parameter
    generate_collage('/mnt/HC_Volume_101955489/gallery-dl/tiktok', './collages', max_images=200)

Collage saved as ./collages/collage_7975.jpg


In [1]:
import os
import random
from PIL import Image
from moviepy.editor import ImageSequenceClip

def generate_frames(root_dir, frame_count=200, output_size=(1920, 1080), images_per_frame=5):
    frames = []
    image_paths = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith('.jpg'):
                image_paths.append(os.path.join(dirpath, filename))

    if not image_paths:
        print("Keine Bilder gefunden!")
        return []

    # Randomize image paths and limit the count
    image_paths = random.sample(image_paths, min(len(image_paths), frame_count * images_per_frame))

    for i in range(frame_count):
        collage = Image.new('RGB', output_size)

        # Select a random subset of images for this frame
        selected_images = random.sample(image_paths, images_per_frame)

        for image_path in selected_images:
            with Image.open(image_path) as img:
                scale_factor = random.uniform(0.2, 0.5)
                new_size = (int(img.size[0] * scale_factor), int(img.size[1] * scale_factor))
                img = img.resize(new_size, Image.LANCZOS)
                
                max_x = max(0, output_size[0] - new_size[0])
                max_y = max(0, output_size[1] - new_size[1])
                pos_x = random.randint(0, max_x)
                pos_y = random.randint(0, max_y)
                
                collage.paste(img, (pos_x, pos_y), img if img.mode == 'RGBA' else None)
        
        frame_path = f'frame_{i}.png'
        frames.append(frame_path)
        collage.save(frame_path)
    
    return frames

def create_video(frames, output_dir, fps=20):
    clip = ImageSequenceClip(frames, fps=fps)
    video_path = os.path.join(output_dir, 'collage_video.mp4')
    clip.write_videofile(video_path, codec='libx264')
    print(f"Video saved as {video_path}")

    # Clean up the frames
    for frame in frames:
        os.remove(frame)

if __name__ == "__main__":
    root_dir = '/mnt/HC_Volume_101955489/gallery-dl/tiktok'
    output_dir = './video_output'
    os.makedirs(output_dir, exist_ok=True)

    # Generate frames
    frames = generate_frames(root_dir, frame_count=200, images_per_frame=5)

    # Create video
    if frames:
        create_video(frames, output_dir, fps=20)

Moviepy - Building video ./video_output/collage_video.mp4.
Moviepy - Writing video ./video_output/collage_video.mp4



Moviepy - Done !
Moviepy - video ready ./video_output/collage_video.mp4
Video saved as ./video_output/collage_video.mp4


In [2]:
import os
import random
from PIL import Image, ImageEnhance
from moviepy.editor import ImageSequenceClip

def blend_images(root_dir, frame_count=200, output_size=(1920, 1080), total_duration=10):
    frames = []
    image_paths = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith('.jpg'):
                image_paths.append(os.path.join(dirpath, filename))

    if not image_paths:
        print("Keine Bilder gefunden!")
        return []

    # Randomly sample the images you need
    selected_images = random.sample(image_paths, min(len(image_paths), frame_count))

    base_collage = Image.new('RGB', output_size, (0, 0, 0))  # Start from a black background

    for idx, image_path in enumerate(selected_images):
        print(f"Processing image {idx + 1} of {frame_count}")
        with Image.open(image_path) as img:
            scale_factor = random.uniform(0.2, 0.5)
            new_size = (int(img.size[0] * scale_factor), int(img.size[1] * scale_factor))
            img = img.resize(new_size, Image.LANCZOS)

            max_x = max(0, output_size[0] - new_size[0])
            max_y = max(0, output_size[1] - new_size[1])
            pos_x = random.randint(0, max_x)
            pos_y = random.randint(0, max_y)

            # Gradually increase opacity
            for opacity in range(1, 11):  # 1 to 10 opacity levels
                enhancer = ImageEnhance.Brightness(img.convert('RGBA'))
                faded_img = enhancer.enhance(opacity / 10.0)
                
                frame = base_collage.copy()
                frame.paste(faded_img, (pos_x, pos_y), faded_img)
                
                frame_path = f'frame_{idx * 10 + opacity}.png'
                frames.append(frame_path)
                frame.save(frame_path)

    return frames

def create_video(frames, output_dir, fps=20):
    clip = ImageSequenceClip(frames, fps=fps)
    video_path = os.path.join(output_dir, 'collage_video.mp4')
    clip.write_videofile(video_path, codec='libx264')
    print(f"Video saved as {video_path}")

    # Clean up the frames
    for frame in frames:
        os.remove(frame)

if __name__ == "__main__":
    root_dir = '/mnt/HC_Volume_101955489/gallery-dl/tiktok'  # Adjust your path
    output_dir = './video_output'
    os.makedirs(output_dir, exist_ok=True)

    # Generate blended frames
    frames = blend_images(root_dir, frame_count=200)

    # Create video
    if frames:
        create_video(frames, output_dir, fps=20)

Processing image 1 of 200
Processing image 2 of 200
Processing image 3 of 200
Processing image 4 of 200
Processing image 5 of 200
Processing image 6 of 200
Processing image 7 of 200
Processing image 8 of 200
Processing image 9 of 200
Processing image 10 of 200
Processing image 11 of 200
Processing image 12 of 200
Processing image 13 of 200
Processing image 14 of 200
Processing image 15 of 200
Processing image 16 of 200
Processing image 17 of 200
Processing image 18 of 200
Processing image 19 of 200
Processing image 20 of 200
Processing image 21 of 200
Processing image 22 of 200
Processing image 23 of 200
Processing image 24 of 200
Processing image 25 of 200
Processing image 26 of 200
Processing image 27 of 200
Processing image 28 of 200
Processing image 29 of 200
Processing image 30 of 200
Processing image 31 of 200
Processing image 32 of 200
Processing image 33 of 200
Processing image 34 of 200
Processing image 35 of 200
Processing image 36 of 200
Processing image 37 of 200
Processing

Moviepy - Done !
Moviepy - video ready ./video_output/collage_video.mp4
Video saved as ./video_output/collage_video.mp4


## 2nd TRY

In [1]:
import os
import random
from PIL import Image, ImageEnhance
from moviepy.editor import ImageSequenceClip

def blend_images(root_dir, frame_count=200, output_size=(1920, 1080), total_duration=10):
    frames = []
    image_paths = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith('.jpg'):
                image_paths.append(os.path.join(dirpath, filename))

    if not image_paths:
        print("Keine Bilder gefunden!")
        return []

    # Randomly sample the images you need
    selected_images = random.sample(image_paths, min(len(image_paths), frame_count))

    base_collage = Image.new('RGB', output_size, (0, 0, 0))  # Start from a black background

    for idx, image_path in enumerate(selected_images):
        print(f"Processing image {idx + 1} of {frame_count}")
        with Image.open(image_path) as img:
            scale_factor = random.uniform(0.2, 0.5)
            new_size = (int(img.size[0] * scale_factor), int(img.size[1] * scale_factor))
            img = img.resize(new_size, Image.LANCZOS)

            max_x = max(0, output_size[0] - new_size[0])
            max_y = max(0, output_size[1] - new_size[1])
            pos_x = random.randint(0, max_x)
            pos_y = random.randint(0, max_y)

            # Gradually increase opacity
            for opacity in range(1, 11):  # 1 to 10 opacity levels
                enhancer = ImageEnhance.Brightness(img.convert('RGBA'))
                faded_img = enhancer.enhance(opacity / 10.0)
                
                frame = base_collage.copy()
                frame.paste(faded_img, (pos_x, pos_y), faded_img)
                
                frame_path = f'frame_{idx * 10 + opacity}.png'
                frames.append(frame_path)
                frame.save(frame_path)

    return frames

def create_video(frames, output_dir, fps=20):
    clip = ImageSequenceClip(frames, fps=fps)
    video_path = os.path.join(output_dir, 'collage_video.mp4')
    clip.write_videofile(video_path, codec='libx264')
    print(f"Video saved as {video_path}")

    # Clean up the frames
    for frame in frames:
        os.remove(frame)

if __name__ == "__main__":
    root_dir = '/mnt/HC_Volume_101955489/gallery-dl/tiktok'  # Adjust your path
    output_dir = './video_output'
    os.makedirs(output_dir, exist_ok=True)

    # Generate blended frames
    frames = blend_images(root_dir, frame_count=200)

    # Create video
    if frames:
        create_video(frames, output_dir, fps=20)

Processing image 1 of 200
Processing image 2 of 200
Processing image 3 of 200
Processing image 4 of 200
Processing image 5 of 200
Processing image 6 of 200
Processing image 7 of 200
Processing image 8 of 200
Processing image 9 of 200
Processing image 10 of 200
Processing image 11 of 200
Processing image 12 of 200
Processing image 13 of 200
Processing image 14 of 200
Processing image 15 of 200
Processing image 16 of 200
Processing image 17 of 200
Processing image 18 of 200
Processing image 19 of 200
Processing image 20 of 200
Processing image 21 of 200
Processing image 22 of 200
Processing image 23 of 200
Processing image 24 of 200
Processing image 25 of 200
Processing image 26 of 200
Processing image 27 of 200
Processing image 28 of 200
Processing image 29 of 200
Processing image 30 of 200
Processing image 31 of 200
Processing image 32 of 200
Processing image 33 of 200
Processing image 34 of 200
Processing image 35 of 200
Processing image 36 of 200
Processing image 37 of 200
Processing

Moviepy - Done !
Moviepy - video ready ./video_output/collage_video.mp4
Video saved as ./video_output/collage_video.mp4


In [3]:
import os
import random
import numpy as np
from PIL import Image
from moviepy.editor import ImageSequenceClip
import logging

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def blend_images(root_dir, frame_count=50, output_size=(1920, 1080), steps_per_image=10):
    frames = []
    image_paths = []

    logging.info(f"Scanning images in {root_dir}.")
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith('.jpg'):
                image_paths.append(os.path.join(dirpath, filename))
                logging.debug(f"Found image: {filename}")

    if not image_paths:
        logging.warning("Keine Bilder gefunden!")
        return []

    selected_images = random.sample(image_paths, min(len(image_paths), frame_count))
    logging.info(f"Selected {len(selected_images)} images for processing.")

    base_collage = Image.new('RGBA', output_size, (0, 0, 0, 255))

    for idx, image_path in enumerate(selected_images):
        logging.info(f"Processing image {idx + 1} of {len(selected_images)}: {image_path}")
        with Image.open(image_path) as img:
            scale_factor = random.uniform(0.5, 0.9)  # Reduce the downscale to preserve more details
            new_size = (int(img.size[0] * scale_factor), int(img.size[1] * scale_factor))
            img = img.resize(new_size, Image.LANCZOS).convert('RGBA')
            logging.debug(f"Resized image to {new_size} with scale factor {scale_factor:.2f}")

            max_x = max(0, output_size[0] - new_size[0])
            max_y = max(0, output_size[1] - new_size[1])
            pos_x = random.randint(0, max_x)
            pos_y = random.randint(0, max_y)
            logging.debug(f"Placing image at position ({pos_x}, {pos_y})")

            # More intermediary steps for smoother transitions
            for opacity in range(1, steps_per_image + 1):
                temp_collage = base_collage.copy()
                faded_img = img.copy()
                alpha = int(255 * (opacity / steps_per_image))
                faded_img.putalpha(alpha)

                temp_collage.paste(faded_img, (pos_x, pos_y), faded_img)
                frames.append(np.array(temp_collage))

            base_collage.paste(img, (pos_x, pos_y), img)

    logging.info("Finished blending images.")
    return frames

def create_video(frames, output_dir, fps=25):
    if not frames:
        logging.warning("No frames to process.")
        return

    logging.info(f"Creating video with {len(frames)} frames.")
    video_path = os.path.join(output_dir, 'collage_video_fo.mp4')
    
    # Increase the fps for smoother animation
    # Increase bitrate using 'write_videofile' to ensure high quality
    video = ImageSequenceClip([frame for frame in frames], fps=fps)
    video.write_videofile(video_path, codec='libx264', bitrate="5000k", threads=4, preset='medium')
    logging.info(f"Video saved as {video_path}")

if __name__ == "__main__":
    root_dir = '/mnt/HC_Volume_101955489/gallery-dl/tiktok'
    output_dir = './video_output'
    os.makedirs(output_dir, exist_ok=True)

    # Generate blended frames
    frames = blend_images(root_dir, frame_count=50)

    # Create video
    if frames:
        create_video(frames, output_dir, fps=25)

2025-02-11 21:50:21,697 - INFO - Scanning images in /mnt/HC_Volume_101955489/gallery-dl/tiktok.
2025-02-11 21:50:21,833 - INFO - Selected 50 images for processing.
2025-02-11 21:50:21,838 - INFO - Processing image 1 of 50: /mnt/HC_Volume_101955489/gallery-dl/tiktok/__afd_2025__/#goviral #deutschland #germany  [7461253154072497430_1_fe665c9648794c03a97ab4f893ce129b].jpg
2025-02-11 21:50:21,999 - INFO - Processing image 2 of 50: /mnt/HC_Volume_101955489/gallery-dl/tiktok/maximilian_.2/#aliceweidel #afd #politik #deutschland  [7408117971857509664_1_986c4eaecc4340768c6a293fd2f38c2f].jpg
2025-02-11 21:50:22,216 - INFO - Processing image 3 of 50: /mnt/HC_Volume_101955489/gallery-dl/tiktok/maximilian_.2/#aliceweidel #afd #politik #deutschland  [7385600163592588577_28_9aaf601b09824f489e2bb7f6b134495c].jpg
2025-02-11 21:50:22,277 - INFO - Processing image 4 of 50: /mnt/HC_Volume_101955489/gallery-dl/tiktok/einervonachtzigmillion/TikTok photo #7318261864729742625 [7318261864729742625_3_f06542b42

Moviepy - Building video ./video_output/collage_video_f.mp4.
Moviepy - Writing video ./video_output/collage_video_f.mp4



2025-02-11 21:50:45,615 - INFO - Video saved as ./video_output/collage_video_f.mp4


Moviepy - Done !
Moviepy - video ready ./video_output/collage_video_f.mp4


In [ ]:
import os
import random
import numpy as np
from PIL import Image
from moviepy.editor import ImageSequenceClip
import logging

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def blend_images(root_dir, frame_count=150, output_size=(1920, 1080), steps_per_image=15):
    frames = []
    image_paths = []

    logging.info(f"Scanning images in {root_dir}.")
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith('.jpg'):
                image_paths.append(os.path.join(dirpath, filename))
                logging.debug(f"Found image: {filename}")

    if not image_paths:
        logging.warning("Keine Bilder gefunden!")
        return []

    selected_images = random.sample(image_paths, min(len(image_paths), frame_count))
    logging.info(f"Selected {len(selected_images)} images for processing.")

    base_collage = Image.new('RGBA', output_size, (0, 0, 0, 255))

    for idx, image_path in enumerate(selected_images):
        logging.info(f"Processing image {idx + 1} of {len(selected_images)}: {image_path}")
        with Image.open(image_path) as img:
            scale_factor = random.uniform(0.5, 0.9)
            new_size = (int(img.size[0] * scale_factor), int(img.size[1] * scale_factor))
            img = img.resize(new_size, Image.LANCZOS).convert('RGBA')
            logging.debug(f"Resized image to {new_size} with scale factor {scale_factor:.2f}")

            max_x = max(0, output_size[0] - new_size[0])
            max_y = max(0, output_size[1] - new_size[1])
            pos_x = random.randint(0, max_x)
            pos_y = random.randint(0, max_y)
            logging.debug(f"Placing image at position ({pos_x}, {pos_y})")

            for opacity in range(1, steps_per_image + 1):
                temp_collage = base_collage.copy()
                faded_img = img.copy()
                alpha = int(255 * (opacity / steps_per_image))
                faded_img.putalpha(alpha)

                temp_collage.paste(faded_img, (pos_x, pos_y), faded_img)
                frames.append(np.array(temp_collage))

            base_collage.paste(img, (pos_x, pos_y), img)

    logging.info("Finished blending images.")
    return frames

def create_video(frames, output_dir, fps=8):
    if not frames:
        logging.warning("No frames to process.")
        return

    logging.info(f"Creating video with {len(frames)} frames.")
    video_path = os.path.join(output_dir, 'collage_video_f.mp4')
    
    video = ImageSequenceClip([frame for frame in frames], fps=fps)
    video.write_videofile(video_path, codec='libx264', bitrate="5000k", threads=4, preset='medium')
    logging.info(f"Video saved as {video_path}")

if __name__ == "__main__":
    root_dir = '/mnt/HC_Volume_101955489/gallery-dl/tiktok'
    output_dir = './video_output'
    os.makedirs(output_dir, exist_ok=True)

    # Generate blended frames
    frames = blend_images(root_dir, frame_count=150)

    # Create video
    if frames:
        create_video(frames, output_dir, fps=8)

2025-02-11 21:54:48,527 - INFO - Scanning images in /mnt/HC_Volume_101955489/gallery-dl/tiktok.
2025-02-11 21:54:48,667 - INFO - Selected 150 images for processing.
2025-02-11 21:54:48,670 - INFO - Processing image 1 of 150: /mnt/HC_Volume_101955489/gallery-dl/tiktok/nujorg3/TikTok photo #7415461031389990177 [7415461031389990177_1_fa3b30fd64c34d4a912904c9bc606d99].jpg
2025-02-11 21:54:48,851 - INFO - Processing image 2 of 150: /mnt/HC_Volume_101955489/gallery-dl/tiktok/wirsinddiealternative/#wirsinddiealternative #fürunserekinder #alternativemode #neuwahlen #zeitfürpatriotismus  [7436094394723224865_1_58d6cf4e13ef4721b0df5f992a4c5cfc].jpg
2025-02-11 21:54:49,072 - INFO - Processing image 3 of 150: /mnt/HC_Volume_101955489/gallery-dl/tiktok/nobody_angelo_/TikTok photo #7427098083034484001 [7427098083034484001_1_b9e0eb40f63f4969b50cc6e96cf010b1].jpg
2025-02-11 21:54:49,173 - INFO - Processing image 4 of 150: /mnt/HC_Volume_101955489/gallery-dl/tiktok/wirsinddiealternative/#viral #afdmitg